In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold

In [2]:
# train has meter readings, which are obviously lacking in test set
train_df = pd.read_pickle("../data/processed/train_data.pkl")
#test_df = pd.read_pickle("../data/processed/test_data.pkl")

In [3]:
train_df.head()

,meter,meter_reading,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,...,weekday,area_per_floor,outlier_square_feet,outlier_area_per_floor,air_temperature_6_lag,air_temperature_24_lag,dew_temperature_6_lag,dew_temperature_24_lag,cloud_coverage_6_lag,cloud_coverage_24_lag
0,0,0.000000,0,8.913550,11.0,NaN,19.406250,NaN,19.406250,0.0,...,4,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.000000,6,11.549431,NaN,NaN,-7.199219,8.0,-10.601562,0.0,...,4,NaN,False,False,16.768229,18.847656,11.118490,10.296224,NaN,NaN
2,0,48.000000,6,11.549431,NaN,NaN,-7.199219,8.0,-10.601562,0.0,...,4,NaN,False,False,17.519531,18.801758,11.302083,10.154460,NaN,NaN
3,0,21.000000,8,9.951611,NaN,NaN,-7.199219,8.0,-10.601562,0.0,...,4,NaN,False,False,18.153646,18.660156,11.402344,9.991862,NaN,NaN
4,2,6609.379883,0,11.960703,NaN,NaN,-7.199219,8.0,-10.601562,0.0,...,4,NaN,False,False,18.703125,18.518229,11.302083,9.783366,NaN,NaN


In [4]:
train_df.shape

(19229841, 24)

In [5]:
train_df.columns

Index(['meter', 'meter_reading', 'primary_use', 'square_feet', 'year_built',
       'floor_count', 'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'relative_humidity', 'air_temp_f',
       'feels_like_temp', 'hour', 'weekday', 'area_per_floor',
       'outlier_square_feet', 'outlier_area_per_floor',
       'air_temperature_6_lag', 'air_temperature_24_lag',
       'dew_temperature_6_lag', 'dew_temperature_24_lag',
       'cloud_coverage_6_lag', 'cloud_coverage_24_lag'],
      dtype='object')

In [6]:
help(GroupKFold)

Help on class GroupKFold in module sklearn.model_selection._split:

class GroupKFold(_BaseKFold)
 |  GroupKFold(n_splits='warn')
 |  
 |  K-fold iterator variant with non-overlapping groups.
 |  
 |  The same group will not appear in two different folds (the number of
 |  distinct groups has to be at least equal to the number of folds).
 |  
 |  The folds are approximately balanced in the sense that the number of
 |  distinct groups is approximately the same in each fold.
 |  
 |  Parameters
 |  ----------
 |  n_splits : int, default=3
 |      Number of folds. Must be at least 2.
 |  
 |      .. versionchanged:: 0.20
 |          ``n_splits`` default value will change from 3 to 5 in v0.22.
 |  
 |  Examples
 |  --------
 |  >>> import numpy as np
 |  >>> from sklearn.model_selection import GroupKFold
 |  >>> X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
 |  >>> y = np.array([1, 2, 3, 4])
 |  >>> groups = np.array([0, 0, 2, 2])
 |  >>> group_kfold = GroupKFold(n_splits=2)
 |  >>> group_

In [7]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([2, 4, 6, 8])
groups = np.array([1, 2, 3, 4])
group_kfold = GroupKFold(n_splits=4)
group_kfold.get_n_splits(X, y, groups)

4

In [15]:
label = train_df.meter_reading
test_df = train_df.drop(columns = ['meter_reading'])

In [16]:
test_df.columns

Index(['meter', 'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'relative_humidity', 'air_temp_f',
       'feels_like_temp', 'hour', 'weekday', 'area_per_floor',
       'outlier_square_feet', 'outlier_area_per_floor',
       'air_temperature_6_lag', 'air_temperature_24_lag',
       'dew_temperature_6_lag', 'dew_temperature_24_lag',
       'cloud_coverage_6_lag', 'cloud_coverage_24_lag'],
      dtype='object')

In [17]:
kfold = KFold(n_splits = 4, shuffle = False)

In [21]:
for train_index, test_index in kfold.split(test_df, label):
    print("TRAIN:", train_index, "TEST", test_index)

TRAIN: [ 4807461  4807462  4807463 ... 19229838 19229839 19229840] TEST [      0       1       2 ... 4807458 4807459 4807460]
TRAIN: [       0        1        2 ... 19229838 19229839 19229840] TEST [4807461 4807462 4807463 ... 9614918 9614919 9614920]
TRAIN: [       0        1        2 ... 19229838 19229839 19229840] TEST [ 9614921  9614922  9614923 ... 14422378 14422379 14422380]
TRAIN: [       0        1        2 ... 14422378 14422379 14422380] TEST [14422381 14422382 14422383 ... 19229838 19229839 19229840]


In [10]:
train_df.hour

0            0
1            0
2            0
3            0
4            0
            ..
19229836    23
19229837    23
19229838    23
19229839    23
19229840    23
Name: hour, Length: 19229841, dtype: category
Categories (24, int64): [0, 1, 2, 3, ..., 20, 21, 22, 23]